In [2]:
import cv2
import cv2
import numpy as np
import glob
import json
import pandas as pd

https://phamdinhkhanh.github.io/2019/11/22/HOG.html

In [3]:
hog = cv2.HOGDescriptor()

SIZE_IMG = (75, 50)

# 1. Khai báo các tham số
cell_size = (8, 8)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins

# 2. Tính toán các tham số truyền vào HOGDescriptor
# winSize: Kích thước của bức ảnh được crop để chia hết cho cell size.
winSize = (SIZE_IMG[0] // cell_size[1] * cell_size[1], SIZE_IMG[1] // cell_size[0] * cell_size[0])
# blockSize: Kích thước của 1 block
blockSize = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
# blockStride: Số bước di chuyển của block khi thực hiện chuẩn hóa histogram bước 3
blockStride = (cell_size[1], cell_size[0])
print('Kích thước bức ảnh crop theo winSize (pixel): ', winSize)
print('Kích thước của 1 block (pixel): ', blockSize)
print('Kích thước của block stride (pixel): ', blockStride)

# 3. Compute HOG descriptor
hog = cv2.HOGDescriptor(_winSize=winSize,
                        _blockSize=blockSize,
                        _blockStride=blockStride,
                        _cellSize=cell_size,
                        _nbins=nbins)

Kích thước bức ảnh crop theo winSize (pixel):  (72, 48)
Kích thước của 1 block (pixel):  (16, 16)
Kích thước của block stride (pixel):  (8, 8)


In [4]:
# cv2.imread(r"Label_data\LIE\fall001.jpg")
# 4. Load ảnh
img = cv2.imread(r"Label_data\LIE\fall001.jpg")
img = cv2.resize(img, SIZE_IMG)
# 5. Tính toán HOG descriptor
hist = hog.compute(img)
print('Kích thước của HOG descriptor: ', hist.shape)

Kích thước của HOG descriptor:  (1440,)


In [10]:
data_lie = glob.glob("./Label_data/LIE/*")
data_sit = glob.glob("./Label_data/SIT/*")
data_stand = glob.glob("./Label_data/STAND/*")

In [62]:
lie = []
for i in data_lie:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    lie.append(hist)

In [63]:
sit = []
for i in data_sit:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    sit.append(hist)

In [64]:
stand = []
for i in data_stand:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    stand.append(hist)

In [65]:
sit = pd.DataFrame(sit)
lie = pd.DataFrame(lie)
stand = pd.DataFrame(stand)

In [66]:
sit['label'] = [0 for i in range(len(sit))]
stand['label'] = [1 for i in range(len(stand))]
lie['label'] = [2 for i in range(len(lie))]

In [71]:
from sklearn.model_selection import train_test_split


In [72]:
train_lie, test_lie = train_test_split(lie, test_size=0.1, random_state=42)
train_stand, test_stand = train_test_split(stand, test_size=0.1, random_state=42)
train_sit, test_sit = train_test_split(sit, test_size=0.2, random_state=42)

In [83]:
train = pd.concat([train_lie, train_stand, train_sit]).reset_index().sample(frac=1).drop(columns=['index'])
test = pd.concat([test_lie, test_stand, test_sit]).reset_index().sample(frac=1).drop(columns=['index'])

In [86]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,label
1245,0.248379,0.017999,0.010341,0.041032,0.369831,0.035309,0.215348,0.079603,0.228593,0.087705,...,0.049552,0.100854,0.135743,0.084649,0.284435,0.192616,0.236717,0.135806,0.082069,1
333,0.156007,0.136185,0.266198,0.036917,0.266198,0.041815,0.266198,0.253761,0.224219,0.062095,...,0.055033,0.074646,0.073501,0.044424,0.087403,0.092727,0.093755,0.057961,0.036020,2
434,0.309197,0.148210,0.032737,0.002419,0.309197,0.005282,0.034887,0.151700,0.309197,0.173313,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1010,0.305877,0.201085,0.185353,0.181066,0.323242,0.020149,0.117143,0.170590,0.276335,0.072613,...,0.145296,0.206006,0.241514,0.202812,0.241514,0.143679,0.090598,0.057604,0.109292,1


In [87]:
X_train = train.drop('label', axis=1)
y_train = train['label']
X_test = test.drop('label', axis=1)
y_test = test['label']

In [88]:
X_train 

,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
1245,0.248379,0.017999,0.010341,0.041032,0.369831,0.035309,0.215348,0.079603,0.228593,0.087705,...,0.043525,0.049552,0.100854,0.135743,0.084649,0.284435,0.192616,0.236717,0.135806,0.082069
333,0.156007,0.136185,0.266198,0.036917,0.266198,0.041815,0.266198,0.253761,0.224219,0.062095,...,0.097588,0.055033,0.074646,0.073501,0.044424,0.087403,0.092727,0.093755,0.057961,0.036020
434,0.309197,0.148210,0.032737,0.002419,0.309197,0.005282,0.034887,0.151700,0.309197,0.173313,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1010,0.305877,0.201085,0.185353,0.181066,0.323242,0.020149,0.117143,0.170590,0.276335,0.072613,...,0.241514,0.145296,0.206006,0.241514,0.202812,0.241514,0.143679,0.090598,0.057604,0.109292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0.314746,0.119778,0.069489,0.023914,0.314746,0.142486,0.036302,0.012235,0.276030,0.073067,...,0.237284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1733,0.191270,0.207102,0.151401,0.110705,0.102248,0.076323,0.118634,0.048736,0.128086,0.235893,...,0.003968,0.014416,0.004028,0.009149,0.000034,0.025649,0.000000,0.001180,0.005423,0.014659
2787,0.314164,0.010048,0.078165,0.093064,0.314164,0.110611,0.089319,0.014775,0.314164,0.179936,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.958134,0.000000,0.000000,0.000000,0.000000
883,0.341163,0.142388,0.341163,0.002965,0.341163,0.048500,0.341163,0.211401,0.341163,0.106018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
y_test.value_counts(),y_train.value_counts()

(label
 0    323
 1    103
 2     79
 Name: count, dtype: int64,
 label
 0    1291
 1     927
 2     711
 Name: count, dtype: int64)

In [92]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [95]:
svm_model = SVC(kernel='poly')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy:  0.9663366336633663
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       323
           1       0.93      0.98      0.95       103
           2       0.95      0.96      0.96        79

    accuracy                           0.97       505
   macro avg       0.95      0.97      0.96       505
weighted avg       0.97      0.97      0.97       505



In [96]:
# Sử dụng mô hình Decision Tree
# Tạo mô hình Decision Tree
dt_model = DecisionTreeClassifier()

# Huấn luyện mô hình trên tập huấn luyện
dt_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = dt_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Hiển thị báo cáo phân loại
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       323
           1       0.83      0.89      0.86       103
           2       0.84      0.89      0.86        79

    accuracy                           0.91       505
   macro avg       0.87      0.90      0.89       505
weighted avg       0.91      0.91      0.91       505



In [97]:
from sklearn.linear_model import LogisticRegression

# Tạo mô hình Logistic Regression
lr_model = LogisticRegression()

# Huấn luyện mô hình trên tập huấn luyện
lr_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = lr_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.96


In [98]:
# Lưu mô hình
import joblib

joblib.dump(svm_model, 'svm_model_hog.pkl')

['svm_model_hog.pkl']

# Test 

In [99]:
loaded_model = joblib.load('svm_model_hog.pkl')

# Dự đoán
y_pred = loaded_model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))


Accuracy:  0.9663366336633663
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       323
           1       0.93      0.98      0.95       103
           2       0.95      0.96      0.96        79

    accuracy                           0.97       505
   macro avg       0.95      0.97      0.96       505
weighted avg       0.97      0.97      0.97       505

